PCB Thermal Analysis using AEDT and Q3D

In [ ]:
import os
import json
import pandas as pd
from pyaedt import Q3d, Edb


# Configure the simulation parameters in this cell


In [1]:

# AEDT Version and Processing Configuration
AEDT_VERSION = "2024.2"
NUM_CORES = 4
NG_MODE = False

# Project Paths
project_folder = "C:/Your/Project/Folder"  # Update this path
edb_file = "C:/Path/To/Your/PCB.aedt"      # Update this path
json_file = "C:/Path/To/buck_converter_harmonics.json"  # Update this path

# Component and Net Configuration
hs_switch = "Q1"     # High-side switch designator
ls_switch = "Q2"     # Low-side switch designator
net_vin = "VIN"      # Net name for input voltage
net_gnd = "GND"      # Net name for ground
net_sw = "SW"        # Net name for switch node


In [ ]:
# Load Harmonic Data from JSON
# This cell loads the harmonic data from the JSON file


def load_harmonic_data(json_file):
    """Load and process harmonic data from JSON file"""
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Extract switch and diode currents from harmonics section
    switch_harmonics = data['harmonics']['switch_current']
    diode_harmonics = data['harmonics']['diode_current']

    # Create a dataframe for the harmonics
    harmonic_data = []
    for i in range(min(len(switch_harmonics), len(diode_harmonics))):
        harmonic_data.append({
            "Frequency": switch_harmonics[i]['frequency_khz'] * 1000,  # Convert to Hz
            "High Side Current": switch_harmonics[i]['magnitude'],
            "Low Side Current": diode_harmonics[i]['magnitude']
        })

    return pd.DataFrame(harmonic_data)

# Load the harmonic data
harmonic_data = load_harmonic_data(json_file)
print("Harmonic data loaded successfully:")
display(harmonic_data.head())

# Open EDB Project and Identify Pins
# This cell opens the EDB project and identifies the relevant component pins

In [ ]:
def identify_pins(edb_file, hs_switch, ls_switch, net_vin, net_gnd, net_sw):
    """Open EDB project and identify component pins"""
    edb = Edb(edb_file, edbversion=AEDT_VERSION)
    print(f"Opened EDB file: {edb_file}")

    # Define function to find pins
    def find_pin(component, net_name):
        return [
            pin for pin in edb.components[component].pins.values()
            if pin.net_name == net_name
        ][0]

    # Identify pins
    hs_pin = find_pin(hs_switch, net_sw)
    ls_pin = find_pin(ls_switch, net_sw)
    vin_pin = find_pin(hs_switch, net_vin)
    gnd_pin = find_pin(hs_switch, net_gnd)

    print(f"HS Switch Pin: {hs_pin.position}")
    print(f"LS Switch Pin: {ls_pin.position}")
    print(f"Vin Pin: {vin_pin.position}")
    print(f"GND Pin: {gnd_pin.position}")

    # Close EDB after pin identification
    edb.save_edb()
    edb.close_edb()

    return hs_pin, ls_pin, vin_pin, gnd_pin

# Identify pins from EDB file
hs_pin, ls_pin, vin_pin, gnd_pin = identify_pins(
    edb_file, hs_switch, ls_switch, net_vin, net_gnd, net_sw
)

In [ ]:
# This cell runs the Q3D analysis for each harmonic frequency

# %%
def run_q3d_analysis(project_folder, harmonic_data, hs_pin, ls_pin, net_vin, net_gnd):
    """Run Q3D simulation for each harmonic frequency"""
    # Open Q3D
    q3d_project_path = os.path.join(project_folder, "Exported_Q3D.aedt")
    q3d = Q3d(q3d_project_path, version=AEDT_VERSION)
    print("Q3D project loaded successfully.")

    # Results container
    results = []

    # Iterate over frequencies
    for _, row in harmonic_data.iterrows():
        freq = row["Frequency"]
        hs_current = row["High Side Current"]
        ls_current = row["Low Side Current"]

        print(f"Running analysis for frequency: {freq} Hz")

        # Set source currents
        q3d.create_source(hs_pin.position, net_name=net_vin, current=f"{hs_current}A")
        q3d.create_sink(ls_pin.position, net_name=net_gnd, current=f"{ls_current}A")

        # Update frequency and analyze
        setup = q3d.create_setup()
        setup.props["SaveFields"] = True
        setup.analyze()

        # Extract temperatures at pin positions
        hs_temp = q3d.post.get_value_at_location(hs_pin.position, "Temperature", setup.name)
        ls_temp = q3d.post.get_value_at_location(ls_pin.position, "Temperature", setup.name)

        results.append({
            "Frequency (Hz)": freq,
            "HS Temp (C)": hs_temp,
            "LS Temp (C)": ls_temp,
        })

    # Release Q3D
    q3d.release_desktop()
    print("Q3D analysis complete and desktop released.")

    return results

# Run Q3D analysis
results = run_q3d_analysis(
    project_folder, harmonic_data, hs_pin, ls_pin, net_vin, net_gnd
)

In [ ]:
# This cell saves the results to a CSV file and creates visualizations

# %%
def save_and_visualize_results(project_folder, results):
    """Save results to CSV and create visualizations"""
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Save results to CSV
    output_csv = os.path.join(project_folder, "DCIR_Temperature_Results.csv")
    results_df.to_csv(output_csv, index=False)
    print(f"Results saved to: {output_csv}")

    # Display results
    display(results_df)

    # Create visualizations
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 6))
    plt.plot(results_df["Frequency (Hz)"]/1000, results_df["HS Temp (C)"], 'r-o', label="High-Side Switch")
    plt.plot(results_df["Frequency (Hz)"]/1000, results_df["LS Temp (C)"], 'b-s', label="Low-Side Switch")
    plt.xlabel("Frequency (kHz)")
    plt.ylabel("Temperature (°C)")
    plt.title("Switch Temperature vs. Frequency")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Save and visualize results
save_and_visualize_results(project_folder, results)

In [ ]:
def run_complete_workflow():
    """Run the complete workflow from loading data to saving results"""
    # 1. Load harmonic data
    harmonic_data = load_harmonic_data(json_file)
    print("Harmonic data loaded successfully.")

    # 2. Identify pins from EDB
    hs_pin, ls_pin, vin_pin, gnd_pin = identify_pins(
        edb_file, hs_switch, ls_switch, net_vin, net_gnd, net_sw
    )

    # 3. Run Q3D analysis
    results = run_q3d_analysis(
        project_folder, harmonic_data, hs_pin, ls_pin, net_vin, net_gnd
    )

    # 4. Save and visualize results
    save_and_visualize_results(project_folder, results)

    print("Complete workflow executed successfully.")

# Uncomment the following line to run the complete workflow
# run_complete_workflow()